In [1]:
import os, time
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from torchsummary import summary
device = "cuda"

In [2]:
import torch.nn as nn
import torch.nn.functional as F
class convNet(nn.Module):
    def __init__(self,in_channels):
        super(convNet,self).__init__()
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
        #                dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=64,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=64,momentum=0.1)
        self.c2 = nn.Conv2d(64,64,3,1,1)
        self.bn2 = nn.BatchNorm2d(num_features=64,momentum=0.1)
        self.m1 = nn.MaxPool2d(2)
        self.d1 = nn.Dropout(0.2)
        
        self.c3 = nn.Conv2d(64,128,3,1,1)
        self.bn3 = nn.BatchNorm2d(128,0.1)
        self.c4 = nn.Conv2d(128,128,3,1,1)
        self.bn4 = nn.BatchNorm2d(128,0.1)
        self.m2 = nn.MaxPool2d(2)
        self.d2 = nn.Dropout(0.2)
        
        self.c5 = nn.Conv2d(128,256,3,1,1)
        self.bn5 = nn.BatchNorm2d(256,0.1)
        self.c6 = nn.Conv2d(256,256,3,1,1)
        self.bn6 = nn.BatchNorm2d(256,0.1)
        self.m3 = nn.MaxPool2d(2)
        self.d3 = nn.Dropout(0.2)
        
        self.fc1 = nn.Linear(256*3*3,256)
        self.fc2 = nn.Linear(256,256)
        self.out = nn.Linear(256,10)

    def forward(self,x):
        x = F.leaky_relu(self.bn1(self.c1(x)),negative_slope=0.1)
        x = F.leaky_relu(self.bn2(self.c2(x)),0.1)
        x = self.m1(x)
        x = self.d1(x)
        
        x = F.leaky_relu(self.bn3(self.c3(x)),0.1)
        x = F.leaky_relu(self.bn4(self.c4(x)),0.1)
        x = self.m2(x)
        x = self.d2(x)
        
        x = F.leaky_relu(self.bn5(self.c5(x)),0.1)
        x = F.leaky_relu(self.bn6(self.c6(x)),0.1)
        x = self.m3(x)
        x = self.d3(x)
        
        x = x.view(-1, 256*3*3) #reshape
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)
                

In [7]:
model = convNet(in_channels=1)
model.cuda()
summary(model, input_size=(1, 28, 28))
# count_parameters(convNet(1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
       BatchNorm2d-2           [-1, 64, 28, 28]             128
            Conv2d-3           [-1, 64, 28, 28]          36,928
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
           Dropout-6           [-1, 64, 14, 14]               0
            Conv2d-7          [-1, 128, 14, 14]          73,856
       BatchNorm2d-8          [-1, 128, 14, 14]             256
            Conv2d-9          [-1, 128, 14, 14]         147,584
      BatchNorm2d-10          [-1, 128, 14, 14]             256
        MaxPool2d-11            [-1, 128, 7, 7]               0
          Dropout-12            [-1, 128, 7, 7]               0
           Conv2d-13            [-1, 256, 7, 7]         295,168
      BatchNorm2d-14            [-1, 25

In [10]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', 'x'.join(str(x) for x in list(param.size())), '=', num_param)
            else:
                print(name, ':', num_param)
            total_param += num_param
    return total_param

In [16]:
trans = transforms.Compose([
#         transforms.RandomResizedCrop(28),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
#         transforms.RandomAffine(180),
        transforms.ToTensor(),  #Take Image as input and convert to tensor with value from 0 to1
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225])
    ])

trans_val = transforms.ToTensor()

class KMnistDataset(Dataset):
    def __init__(self,data_len=None, is_validate=False,validate_rate=None):
        self.is_validate = is_validate
        self.data = pd.read_csv("./dataset/train.csv")
        print("data shape:", np.shape(self.data))
        if data_len == None:
            data_len = len(self.data)
        
        if self.is_validate:
            self.len = int(data_len*validate_rate)
            self.offset = int(data_len*(1-validate_rate))
            self.transform = trans_val
        else:
            self.len = int(data_len*(1-validate_rate))
            self.offset = 0
            self.transform = trans
        
    def __getitem__(self, idx):
        idx += self.offset
#         if self.offset !=0:
#             print(idx)
        img = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
        label = self.data.iloc[idx, 0]  #(num,)
        img = Image.fromarray(img)

        img = self.transform(img)     #value: 0~1, shape:(1,28,28)
        label = torch.as_tensor(label, dtype=torch.uint8)    #value: 0~9, shape(1)
            
        return img, label

    def __len__(self):
        return self.len

In [17]:
batch_size = 64
num_workers = 8
# kmnist_dataset = datasets.ImageFolder(kmnist_dataset, transform=None)

vr = 0.1
train_dataset = KMnistDataset(data_len=None,is_validate=False, validate_rate=vr)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

validate_dataset = KMnistDataset(data_len=None,is_validate=True, validate_rate=vr)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


data shape: (60000, 785)
data shape: (60000, 785)


In [ ]:
kmnist_dataset.data.head(100)

In [ ]:
type(kmnist_dataset.data.iloc[20,1:].values)  #numpy ndarray
type(kmnist_dataset.data.iloc[20,0])  #numpy int64
kmnist_dataset.data.head(5)

In [21]:
def main():
    epochs = 200
    max_acc = 0
    data_num = 0
    loss_avg = 0
    lr = 1e-3
    validate_ep = 10
    
    criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.SGD(model.parameters(),lr=lr)
#     optimizer = torch.optim.SGD(net_Momentum.parameters(),lr=Learning_rate,momentum=0.8,nesterov=True)
#     optimizer = torch.optim.RMSprop(model.parameters(),lr=lr,alpha=0.9)
#     optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=(0.9,0.99))
    optimizer = torch.optim.Adagrad(model.parameters(),lr=lr)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=3)
    
    for ep in range(0,epochs):
        model.train()
        for idx, data in enumerate(train_loader):
            img, target = data
            img, target = img.to(device), target.to(device,dtype=torch.long)
#             print(np.shape(img),np.shape(target)) #Tensor(4,1,28,28), Tensor(4)
#             print(np.max(img.cpu().numpy()),np.min(img.cpu().numpy())) #1.0 0.0
            pred = model(img)
#             print(pred.size())   #(32,10)
#             print(target.size()) #(32,)
            ###Input shape must be pred:, target:
            loss = criterion(pred,target)
        
            loss_avg += loss.item()
            data_num += img.size(0)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        ###Evaluate Train Loss 
        if ep%2 == 0:
            loss_avg /= data_num
            print("Ep:{}, loss:{}".format(ep,loss_avg))
            loss_avg = 0
            data_num = 0
        
        ###Validation
        if ep!=0 and ep%validate_ep == 0:
            model.eval()
            acc = 0
            val_loss = 0
            data_num  = 0
            with torch.no_grad():
                for idx, data in enumerate(validate_loader):
                    img, target = data
                    img, target = img.to(device), target.to(device,dtype=torch.long)
                    pred = model(img)
                    val_loss += criterion(pred, target)
                    # print(pred)
                    _,pred_class = torch.max(pred.data, 1)
#                     print(pred_class)
                    acc += (pred_class == target).sum().item()
                    data_num += img.size(0)
                lr_scheduler.step(val_loss)
            
            acc /= data_num
            val_loss /= data_num
            if acc > max_acc:
                max_acc = acc
                if max_acc > 0.95 :
                    print("===================Best Model Saved:==================")                    
                    torch.save(model.state_dict(), "./Kmnist_saved_model/improved_net/ep{}_acc{:.4f}".format(ep,max_acc))
                    print("======================================================")
            print("Episode:{}, Validation Loss:{}, Acc:{:.4f}%".format(ep,val_loss,max_acc*100))
            
if __name__ == "__main__":
    main()
    

Ep:0, loss:0.0009851861679926516
Ep:2, loss:0.0008459037060182945
Ep:4, loss:0.0007879626771512752
Ep:6, loss:0.0007500532162828474
Ep:8, loss:0.0008116703499798421
Ep:10, loss:0.000725715799729288
===================Best Model Saved:==================
Episode:10, Validation Loss:0.000918528123293072, Acc:98.5667%
Ep:12, loss:0.000748796961735934
Ep:14, loss:0.0007210418479236843
Ep:16, loss:0.0007044634509510177
Ep:18, loss:0.000773176575446053
Ep:20, loss:0.0007450611972319031
===================Best Model Saved:==================
Episode:20, Validation Loss:0.0008697066805325449, Acc:98.6000%
Ep:22, loss:0.0007174018820014521
Ep:24, loss:0.0006625599828333146
Ep:26, loss:0.0007378380852406499
Ep:28, loss:0.0006810620519574042
Ep:30, loss:0.0007435431647830194
===================Best Model Saved:==================
Episode:30, Validation Loss:0.0008088580798357725, Acc:98.7333%
Ep:32, loss:0.0006294088053318933
Ep:34, loss:0.0006703107081940053
Ep:36, loss:0.0006605172483153486
Ep:38,